In [18]:
import pandas as pd

data = pd.read_csv('./data/avazu-ctr-prediction/train', nrows = 300000)
data.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [19]:
print(data.columns)
print(data['click'].value_counts())

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')
0    248789
1     51211
Name: click, dtype: int64


In [20]:
data = data.drop(['id'], axis=1)

In [42]:
print(data['site_id'].value_counts())
print(pd.factorize(data['site_id']))

0       92032
7       78840
10      13580
8       12573
4        9089
        ...  
1392        1
880         1
1391        1
1330        1
929         1
Name: site_id, Length: 1456, dtype: int64
(array([  0,   0,   0, ..., 151,   0,   0]), Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455],
           dtype='int64', length=1456))


In [22]:
for feature in data.drop('click', axis=1).columns:
    data[feature] = pd.factorize(data[feature])[0]
data.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,1,1,1,0,0,0,...,0,1,2,0,0,1,0,0,0,1


In [43]:
# 切分训练集，测试集
from sklearn.model_selection import train_test_split
train, valid = train_test_split(data, test_size=0.2)

In [49]:
catFeatures = list(train.drop('click', axis=1).columns)
catFeatures

['hour',
 'C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21']

In [51]:
# 使用LightGBM
import lightgbm as lgb
clf = lgb.LGBMClassifier(is_unbalanced=False, slient=False)

clf.fit(train.drop('click', axis=1), train['click'], categorical_feature=catFeatures, verbose=5)
predictProba = clf.predict_proba(valid.drop('click', axis=1))
predict = predictProba[:, 1]

predictProba, predict 

(array([[0.70327916, 0.29672084],
        [0.985156  , 0.014844  ],
        [0.81118711, 0.18881289],
        ...,
        [0.99462332, 0.00537668],
        [0.75014603, 0.24985397],
        [0.87679141, 0.12320859]]),
 array([0.29672084, 0.014844  , 0.18881289, ..., 0.00537668, 0.24985397,
        0.12320859]))

In [52]:
# 打印二分类交叉熵
from sklearn.metrics import log_loss
print(log_loss(valid['click'], predict))


0.3936567155484958


In [54]:
test = pd.read_csv('./data/avazu-ctr-prediction/test')
test = test.drop(['id'], axis=1)

for feature in test.columns:
    test[feature] = pd.factorize(test[feature])[0]
    
test.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0,...,0,0,1,0,0,1,1,1,1,1
2,0,0,0,1,1,1,0,0,0,0,...,0,0,1,0,0,1,1,1,1,1
3,0,0,0,2,2,2,1,1,1,0,...,0,0,2,0,0,2,0,2,2,2
4,0,0,0,2,2,2,2,2,2,0,...,0,0,3,0,0,3,1,3,3,3


In [ ]:
predictTest = clf.predict_proba(test)[:, 1]
